In [10]:
import cv2
import dlib
import numpy as np


PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(PREDICTOR_PATH)
#cascade_path='haarcascade_frontalface_default.xml'
#cascade = cv2.CascadeClassifier(cascade_path)
detector = dlib.get_frontal_face_detector()


def get_landmarks(im):
    rects = detector(im, 1)

    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    #Overlays the landmark points on the image itself
    
    im = im.copy()
    for idx, point in enumerate(landmarks):
       pos = (point[0, 0], point[0, 1])
       cv2.putText(im, str(idx), pos,
                   fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                fontScale=0.4,
                  color=(0, 0, 255))
       cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

def top_lip(landmarks):
    top_lip_pts = []
    for i in range(50,53):
        top_lip_pts.append(landmarks[i])
    for i in range(61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:,1])
def left_eye(landmarks):
    left_point=[]
    for i in range(36,38):
        left_point.append(landmarks[i])
    left_all_points = np.squeeze(np.asarray(left_point))
    left_point_mean = np.mean(left_all_points, axis=0)  
    return int(left_point_mean[1])
def left_bottom_eye(landmarks):
    left_bottom_point=[]
    for i in range(39,41):
        left_bottom_point.append(landmarks[i])
    left_bottom_all_points = np.squeeze(np.asarray(left_bottom_point))
    left_bottom_point_mean = np.mean(left_bottom_all_points, axis=0)  
    return int(left_bottom_point_mean[1])    
def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])

def mouth_open(image):
    landmarks = get_landmarks(image)
    
    if landmarks == "error":
        return image, 0,0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks) #upper lip(lop and bottom) 
                                        #lower lip(lop and bottom) 
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)
    #image_with_landmarks = annotate_landmarks(image, landmarks)
    left_top = left_eye(landmarks) #upper lip(lop and bottom) 
                                        #lower lip(lop and bottom) 
    left_bottom = left_bottom_eye(landmarks)
    eyes_distance = abs(left_top - left_bottom)
    return image_with_landmarks, lip_distance,eyes_distance
#def eyes_close(image):
 #   landmarks = get_landmarks(image)
    
 #   if landmarks == "error":
  #      return image, 0
    
  #  image_with_landmarks = annotate_landmarks(image, landmarks)
   # left_top = left_eye(landmarks) #upper lip(lop and bottom) 
                                        #lower lip(lop and bottom) 
   # left_bottom = left_bottom_eye(landmarks)
   # eyes_distance = abs(left_top - left_bottom)
   # return image_with_landmarks, eyes_distance


cap = cv2.VideoCapture(0)
yawns = 0
yawn_status = False 

while True:
    ret, frame = cap.read()   
    image_landmarks,lip_distance,eyes_distance = mouth_open(frame)
    #image_landmarks,eyes_distance=eyes_close(frame)
    prev_yawn_status = yawn_status  
    if eyes_distance>5:
        cv2.putText(frame, "Subject eyes open", (100,450), 
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0),3)
    else:
        cv2.putText(frame, "Subject eyes close", (100,450), 
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,255),3)
    if lip_distance > 20:
        yawn_status = True 
        
        cv2.putText(frame, "Subject is Yawning", (50,150), 
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,0,255),2)
        

        output_text = " Yawn Count: " + str(yawns + 1)

        cv2.putText(frame, output_text, (50,50),
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)
        
    else:
        yawn_status = False 
         
    if prev_yawn_status == True and yawn_status == False: # this counts actual yawns
        yawns += 1
   
    cv2.imshow('Live Landmarks', image_landmarks )
    cv2.imshow('Yawn and eyes Detection', frame )
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

C:\Users\PRAKHAR .LAPTOP-5EKIAHKD\Anaconda3\envs\prem\lib\site-packages\ipykernel_launcher.py:72: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
